In [ ]:
import pandas as pd

In [ ]:
from scipy.stats import spearmanr

df = list()

for i in snakemake.input:
    _df = pd.read_csv(i)
    params = i.replace('.csv', '').split('/')[-1].split('_')
    model = params[0].split('=')[1].replace('e', '')
    lora_rank, lora_layers = params[3].split('=')[1].split(';')
    lora_rank = int(lora_rank)
    stats = spearmanr(_df['Predicted Melting Point'], _df['True Melting Point'])
    df.append({
        'method': i.split('=')[1].split('_')[0],
        'lora_rank': lora_rank,
        'lora_layers': lora_layers,
        'model': model,
        'Correlation': stats.correlation,
    })

df = pd.DataFrame(df)
df.to_csv(snakemake.output['stats'], index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 3, figsize=(10, 5), sharex=True, sharey=True)

for i, (model, df_lora) in enumerate(df.groupby('lora_rank')):
    for j, (lora_rank, df_model) in enumerate(df_lora.groupby('model')):
        sns.scatterplot(data=df_model, y='lora_layers', x='Correlation', ax=axs[i, j])
        axs[i, j].set_title(f'Model: {model}, Lora Rank: {lora_rank}', fontsize=10)
        if (i == 1) and (j == 0):
            axs[i, j].set_ylabel('Lora Layer Configuration')
        elif (i == 2) and (j == 1):
            axs[i, j].set_xlabel('Spearman Correlation')
        else:
            axs[i, j].set_xlabel(None)
            axs[i, j].set_ylabel(None)

plt.savefig(snakemake.output['fig'], bbox_inches='tight', dpi=300, transparent=True)